In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from PIL import Image

In [6]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# ✅ Callback: Dừng khi đạt val_accuracy >= target
class StopAtAccuracy(Callback):
    def __init__(self, target=0.82, save_path="final_model_valaccuracy1.0.keras"):
        super(StopAtAccuracy, self).__init__()
        self.target = target
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get("val_accuracy")
        if val_acc is not None and val_acc >= self.target:
            print(f"\n✅ Đạt val_accuracy = {val_acc:.4f}. Lưu toàn bộ mô hình và dừng huấn luyện.")
            self.model.save(self.save_path)
            self.model.stop_training = True

# Load mô hình đã huấn luyện trước đó
model = load_model("full_model_ContinueTrain.keras")
print("📐 Input shape của model:", model.input_shape)

# Nếu model yêu cầu nhiều channel hơn
expected_channels = model.input_shape[-1]
if expected_channels != 1:
    def repeat_channel(x):
        return np.repeat(x, expected_channels, axis=-1)
    preprocessing_fn = repeat_channel
    print(f"💡 Dữ liệu sẽ được nhân channel từ 1 lên {expected_channels}.")
else:
    preprocessing_fn = None

# Cho phép fine-tune toàn bộ layers
for layer in model.layers:
    layer.trainable = True


# ImageDataGenerator với augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    preprocessing_function=preprocessing_fn
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    preprocessing_function=preprocessing_fn
)

# Tạo generators
train_generator = train_datagen.flow_from_directory(
    'D:/DatasetDoAnCoSO/dataset_emotion/images/train',
    target_size=(56, 56),
    color_mode='grayscale' if expected_channels == 1 else 'rgb',
    batch_size=16,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    'D:/DatasetDoAnCoSO/dataset_emotion/images/validation',
    target_size=(56, 56),
    color_mode='grayscale' if expected_channels == 1 else 'rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Class weight để hỗ trợ lớp khó
class_weight = {0: 2.0, 1: 2.5, 2: 3.0, 3: 1.0, 4: 1.8, 5: 3.0, 6: 1.0}

# Callbacks
checkpoint = ModelCheckpoint("model_weights_continue.keras", monitor='val_accuracy', save_best_only=True, verbose=1, mode='max')
stop_at_target = StopAtAccuracy(target=0.82, save_path="final_model_valaccuracy_continue.keras")
stop_at_acc_additional = StopAtAccuracy(target=0.82, save_path="final_model_valaccuracy1.0.keras")

callbacks_list = [checkpoint, stop_at_target, stop_at_acc_additional]

# Train
steps_per_epoch = int(np.ceil(train_generator.n / train_generator.batch_size))
validation_steps = int(np.ceil(val_generator.n / val_generator.batch_size))

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=1000,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks_list,
    class_weight=class_weight
)

# Lưu mô hình cuối cùng
model.save("final_model_CNN.keras")


📐 Input shape của model: (None, 56, 56, 1)


KeyboardInterrupt: 